In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-07-15 11:42:03--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-07-15 11:42:03--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-07-15 11:42:03--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/training.1600000.processed.noemoticon.csv' ,encoding = 'latin' ,header = None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
## renaming and removing unnecesary columns
df.columns = ['sentiments', 'id' , 'date' ,'query' ,'username' ,'text']
df.drop(['id','date','query','username'] , axis = 1, inplace = True)
df.head()

,sentiments,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
## checking for skewed datannn
df.sentiments.value_counts()

0    800000
4    800000
Name: sentiments, dtype: int64

In [ ]:
##creating a subset due to limited computational capacity
df_pos = df[df['sentiments'] == 4]
df_neg = df[df['sentiments'] == 0]

num_samples = 50000

df_pos_downsampled = df_pos.sample(num_samples , random_state=2023)
df_neg_downsampled = df_neg.sample(num_samples , random_state=2023)
df_small = pd.concat([df_pos_downsampled , df_neg_downsampled])
df_small.head(10)

,sentiments,text
1381645,4,[Snoop Dogg - Gin 'N Juice] *raise glass to @A...
1021869,4,@SashiGamali lol i want all day everywhere lol
1345323,4,@rebeccablackett ahaaa here comes the aeropla...
1503851,4,@kricket_rc234 what do you need my last name f...
1065908,4,Actually i'm gonna Follow Friday @Kiiiiirsty t...
938164,4,cannot stop smiling today!
1279718,4,"and suspension by mae. fuck, this song will al..."
1527220,4,@gunjansatija will upload soon... and you can'...
1385768,4,@RhythmHippy thank you for the #followfriday
939649,4,chocolatee ice-cream Roocckss!!! i'm just eati...


In [ ]:
lab_to_sentiment = {0 : 0 , 4 : 1}
def label_decoder(label):
  return lab_to_sentiment[label]
df_small.sentiments = df_small.sentiments.apply(lambda x : label_decoder(x))
df_small.head()

,sentiments,text
1381645,1,[Snoop Dogg - Gin 'N Juice] *raise glass to @A...
1021869,1,@SashiGamali lol i want all day everywhere lol
1345323,1,@rebeccablackett ahaaa here comes the aeropla...
1503851,1,@kricket_rc234 what do you need my last name f...
1065908,1,Actually i'm gonna Follow Friday @Kiiiiirsty t...


In [ ]:
import re
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preprocess_text(text):
  text = re.sub(text_cleaning_re,' ',str(text).lower()).strip()
  tokens = []
  for token in text.split():
    tokens.append(token)
  return " ".join(tokens)

df_small.text = df_small.text.apply(lambda x : preprocess_text(x))
## verifying that text is fit for tokenization and embedding
df_small.head()

,sentiments,text
1381645,1,snoop dogg gin n juice raise glass to alonis
1021869,1,lol i want all day everywhere lol
1345323,1,ahaaa here comes the aeroplane xxx
1503851,1,what do you need my last name for
1065908,1,actually i m gonna follow friday kiiiiirsty to...


In [ ]:
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file='glove.6B.100d.txt'
glove_output_file='word.txt'

glove2word2vec(glove_input_file,glove_output_file)

from gensim.models import KeyedVectors
GloVe=KeyedVectors.load_word2vec_format(glove_output_file,binary=False)

<ipython-input-9-236c5e57bf1f>:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file,glove_output_file)


In [ ]:
text = df_small.text.values
labels = df_small.sentiments.values

In [ ]:
## tokenize the words
import tensorflow as tf
from keras.preprocessing.text import one_hot,Tokenizer

word_tokenizer=Tokenizer()
word_tokenizer.fit_on_texts(text)
tokenized_text = word_tokenizer.texts_to_sequences(text)

In [ ]:
print(tokenized_text[0])
print('Max sentence length: ', max([len(sen) for sen in tokenized_text]))


[9449, 9450, 8467, 237, 1746, 5379, 1441, 2, 22522]
Max sentence length:  39


In [ ]:
## Padding
from keras.utils import pad_sequences
MAX_LEN = 48

tokenized_text = pad_sequences(tokenized_text , maxlen = MAX_LEN ,dtype = "long" , value = 0, padding = "post")
print('\Done')

\Done


In [ ]:
print(len(tokenized_text[0]))

48


In [ ]:
##get words from index of tokenizer and get vocab size -> for all words get glove embedding and then proceed
words = word_tokenizer.word_index.keys()
word_list = []
for word in words:
  word_list.append(word)
vocab_size = len(word_list)
print(vocab_size)

57785


In [ ]:
import numpy as np
d_model = 100
embedding_matrix = np.zeros((vocab_size + 1, d_model))

for word in word_list:
  if word in GloVe.key_to_index:
    vector = GloVe[word]
  if vector is not None:
    index = word_tokenizer.word_index[word]
    embedding_matrix[index] = vector
embedding_matrix.shape

(57786, 100)

In [ ]:
from sklearn.model_selection import train_test_split

train_inputs,test_inputs,train_labels,test_labels = train_test_split(tokenized_text,labels,random_state=2023,test_size = 0.15)


In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam,AdamW
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, Flatten, Embedding
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler

lstm_model = Sequential()
## embedding layer takes size of embedding matrix i.e vocab_sizexd_model, input_length = length of sent(max), in lstm units = dimension of hidden state
word_embedding_layer  = Embedding(vocab_size+1, d_model, weights=[embedding_matrix], input_length = MAX_LEN, trainable=False)
lstm_model.add(word_embedding_layer)
lstm_model.add(Bidirectional(LSTM(units = 96, return_sequences = True), input_shape=[MAX_LEN,d_model]))
lstm_model.add(Flatten())
lstm_model.add(Dense(units=1, activation='sigmoid'))


In [ ]:
optimizer = AdamW(weight_decay = 0.01)
##patience = wait for epochs and factor is to reduce the LR gradually
lr_scheduler = ReduceLROnPlateau(factor=0.1, patience=3)

lstm_model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['acc'])
print(lstm_model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 48, 100)           5778600   
                                                                 
 bidirectional (Bidirectiona  (None, 48, 192)          151296    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 1)                 9217      
                                                                 
Total params: 5,939,113
Trainable params: 160,513
Non-trainable params: 5,778,600
_________________________________________________________________
None


In [ ]:
lstm_history=lstm_model.fit(train_inputs,train_labels,batch_size=64,epochs=10,verbose=1,validation_split=0.2)

Epoch 1/10
1063/1063 [==============================] - 108s 99ms/step - loss: 0.5490 - acc: 0.7182 - val_loss: 0.5030 - val_acc: 0.7518
Epoch 2/10
1063/1063 [==============================] - 105s 99ms/step - loss: 0.4900 - acc: 0.7612 - val_loss: 0.4833 - val_acc: 0.7678
Epoch 3/10
1063/1063 [==============================] - 106s 100ms/step - loss: 0.4554 - acc: 0.7834 - val_loss: 0.4751 - val_acc: 0.7708
Epoch 4/10
1063/1063 [==============================] - 107s 100ms/step - loss: 0.4245 - acc: 0.8036 - val_loss: 0.4853 - val_acc: 0.7732
Epoch 5/10
1063/1063 [==============================] - 104s 97ms/step - loss: 0.3913 - acc: 0.8228 - val_loss: 0.4900 - val_acc: 0.7775
Epoch 6/10
1063/1063 [==============================] - 108s 101ms/step - loss: 0.3544 - acc: 0.8426 - val_loss: 0.5041 - val_acc: 0.7705
Epoch 7/10
1063/1063 [==============================] - 104s 98ms/step - loss: 0.3145 - acc: 0.8638 - val_loss: 0.5381 - val_acc: 0.7615
Epoch 8/10
1063/1063 [================

In [ ]:
y_pred=lstm_model.predict(test_inputs)

469/469 [==============================] - 8s 16ms/step


In [ ]:
predictions = []
for pred in y_pred:
  if pred > 0.5:
    predictions.append(1)
  else :
    predictions.append(0)

In [ ]:
from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(predictions,test_labels))

0.5005687282991598
